In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

In [7]:
def configure_browser(driver, url):
    driver.get(url)
    driver.set_window_size(840, 1080)
    driver.implicitly_wait(2)
    
def prepare_website(driver):
#     for remove cookies, ads and rodo shit
    pass

def get_skills(driver):
    skills = driver.find_elements_by_css_selector(".css-1ikoimk > div")
    skills_dict = {}
    for skill in skills:
        skills_dict[skill.find_element_by_css_selector(".css-1eroaug").text] = len(skill.find_elements_by_css_selector(".css-18cijgm"))  
    return skills_dict

def get_salary(driver):
    salary = driver.find_element_by_css_selector(".css-1r1n80v").text
    if "Undisclosed" in salary:
        min_salary = "None"
        max_salary = "None"
        currency = "None"
    else:
        min_value = salary.split("-")
        salary = min_value[-1].split() 
        if len(min_value) > 1:
            min_salary = min_value[0]
        else:
            min_salary = "".join(salary[:2])
        max_salary = "".join(salary[:2])
        currency = salary[-2]
    return min_salary, max_salary, currency

def get_level(driver):
    levels = {
        "senior": 3,
        "mid": 2,
        "junior": 1
    }
    main_level = driver.find_element_by_css_selector("div.css-1uvpahd:nth-child(4) div:nth-child(2)").text
    return levels[main_level]

def get_address(driver):
    address = driver.find_element_by_css_selector(".css-1ue58kn").text.split()
    street = "None"
    if len(address) > 1:
        street = " ".join(address[:-1])  # usun przecinek
    city = address[-1]
    return city, street

def get_offer_details(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".css-34e4ux")))
    offer_name = driver.find_element_by_css_selector(".css-34e4ux").text
    company_name = driver.find_element_by_css_selector("div.css-1uvpahd:nth-child(1) a").text
    company_size = driver.find_element_by_css_selector("div.css-1uvpahd:nth-child(2) div:nth-child(2)").text
    description = driver.find_element_by_css_selector(".css-alatv1").text
    emp_type = driver.find_element_by_css_selector("div.css-1uvpahd:nth-child(3) div:nth-child(2)").text
    city, street = get_address(driver)
    min_salary, max_salary, currency = get_salary(driver)
    main_level = get_level(driver)
    skills = get_skills(driver)                                                                                  
    d = {
        "offer_name": offer_name,
        "company_name": company_name,
        "company_size": company_size,
        "city": city,
        "street": street,
        "min_salary": min_salary,
        "max_salary": max_salary,
        "currency": currency,
        "emp_type": emp_type,
        "main_level": main_level,
        "skills": skills,
        "description": description    }

    return d

def back_to_homepage(driver):
    back_to_home =  driver.find_element_by_css_selector("div > button.MuiButtonBase-root.MuiIconButton-root.css-1j6sxc1")
    back_to_home.click()

def scroll_offers(driver, idx):
    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".css-ic7v2w")))
        offer = driver.find_element_by_css_selector(".css-ic7v2w")
#         offer_name = offer.find_element_by_css_selector('.css-1x9zltl').text
        driver.execute_script(f"arguments[0].scrollTo(0, {idx * 98});", offer)
    except:
        driver.close()

def go_to_offer(driver, idx):
    driver.implicitly_wait(5)
    time.sleep(2)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".css-ic7v2w > div > div:nth-child(2)")))
    offer = driver.find_elements_by_css_selector(".css-ic7v2w > div > div")[0]
    offer.click()
#     webdriver.ActionChains(driver).move_to_element(offer).click(offer).perform()
    
def run_scrapping(driver):
    offers = []
    for idx in range(10):
        go_to_offer(driver, idx)
        offer = get_offer_details(driver)
        offers.append(offer)
        back_to_homepage(driver)
        scroll_offers(driver, idx)
        
    return offers

In [8]:
driver = webdriver.Chrome('./chromedriver')
url = "https://justjoin.it/all/all"

configure_browser(driver, url)
prepare_website(driver)

offers = run_scrapping(driver)

df = pd.DataFrame(offers)
driver.close()

TimeoutException: Message: 


In [9]:
df

NameError: name 'df' is not defined

In [206]:
from datetime import datetime

In [207]:
df.to_csv(f"job_offers-{datetime.now()}.csv")